In [78]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

In [79]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [80]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

In [81]:
def discriminator(bottom, reuse=False):
    
    #print bottom
    dis1 = slim.convolution2d(bottom,16,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,32,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,64,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis3),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

In [107]:
tf.reset_default_graph()

#z_size = 100 #Size of z vector used for generator.

image_size = 256

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)
#fake_in2 = tf.placeholder(tf.float32, shape=[None, 32, 32, 1])
#These two placeholders are used for input into the generator and discriminator, respectively.
#z_in = tf.placeholder(shape=[256,256],dtype=tf.float32) #Random vector
fake_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images

#Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(fake_in,reuse=True) #Produces probabilities for generator images



#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx)+ tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.




In [108]:
tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0001,beta1=0.5)
#trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[:]) #Only update the weights for the discriminator network.
#g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
#update_G = trainerG.apply_gradients(g_grads)

In [109]:
batch_size = 256 #Size of image batch to apply at each iteration.
iterations = 250 #Total number of iterations to use.
sample_directory = './figs' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [134]:
from PIL import Image, ImageOps
img = Image.open("mnist.jpg")


In [135]:
#img.size
grayscale = ImageOps.grayscale(img)
grayscale = grayscale.resize((32,32))
grayscale.size
#img = img.resize((209,209))

(32, 32)

In [136]:
cat_data = np.asarray(grayscale, dtype="float32")

In [137]:
cat_data.shape
cat_data = cat_data.reshape(1,32,32,1)

In [138]:
print cat_data.shape

(1, 32, 32, 1)


In [119]:
with tf.Session() as sess:  
    sess.run(init)
    for i in range(251):
        #zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        xs,_ = mnist.train.next_batch(batch_size) #Draw a sample batch from MNIST dataset.
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 32x32
        D_g,D_x,_,Dloss = sess.run([Dg,Dx,update_D,d_loss],feed_dict={fake_in:cat_data,real_in:xs}) #Update the discriminator
        #print dLoss
        #print D_x[0][0]
        #print D_g
        print "Real prob"
        print D_x[30]
        print "Fake prob"
        print D_g
        if i % 250 == 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')    
            #save_path = saver.save(sess, "models/pretrained_gan.ckpt", global_step=i)
            print("saved to %s" % save_path)
    #save_path = saver.save(sess, "/model.ckpt")
    #print("Model saved in file: %s" % save_path)    
        #_,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs}) #Update the generator, twice for good measure.
        #_,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})

Real prob
[ 0.48096788]
Fake prob
[[ 0.42412743]]
saved to models/pretrained_gan.ckpt-250
Real prob
[ 0.46882528]
Fake prob
[[ 0.32025218]]
Real prob
[ 0.48802608]
Fake prob
[[ 0.24250579]]
Real prob
[ 0.63797683]
Fake prob
[[ 0.18659686]]
Real prob
[ 0.65213656]
Fake prob
[[ 0.14852402]]
Real prob
[ 0.65537548]
Fake prob
[[ 0.12289307]]
Real prob
[ 0.65982401]
Fake prob
[[ 0.10509299]]
Real prob
[ 0.76219404]
Fake prob
[[ 0.09149051]]
Real prob
[ 0.72036076]
Fake prob
[[ 0.0807682]]
Real prob
[ 0.76383454]
Fake prob
[[ 0.0719007]]
Real prob
[ 0.7248289]
Fake prob
[[ 0.06447683]]
Real prob
[ 0.7952103]
Fake prob
[[ 0.05803008]]
Real prob
[ 0.69584543]
Fake prob
[[ 0.05261444]]
Real prob
[ 0.85026848]
Fake prob
[[ 0.04790008]]
Real prob
[ 0.86302018]
Fake prob
[[ 0.04373728]]
Real prob
[ 0.79141498]
Fake prob
[[ 0.04009823]]
Real prob
[ 0.88444835]
Fake prob
[[ 0.03684027]]
Real prob
[ 0.87088138]
Fake prob
[[ 0.03394325]]
Real prob
[ 0.9337858]
Fake prob
[[ 0.0313636]]
Real prob
[ 0.95

Real prob
[ 0.99669743]
Fake prob
[[ 0.00110255]]
Real prob
[ 0.99529582]
Fake prob
[[ 0.00109176]]
Real prob
[ 0.99760664]
Fake prob
[[ 0.00108114]]
Real prob
[ 0.99649423]
Fake prob
[[ 0.00107079]]
Real prob
[ 0.99593192]
Fake prob
[[ 0.00106052]]
Real prob
[ 0.997352]
Fake prob
[[ 0.00105048]]
Real prob
[ 0.99809653]
Fake prob
[[ 0.00104054]]
Real prob
[ 0.99851054]
Fake prob
[[ 0.00103069]]
Real prob
[ 0.99782854]
Fake prob
[[ 0.00102089]]
Real prob
[ 0.99436259]
Fake prob
[[ 0.00101118]]
Real prob
[ 0.99761009]
Fake prob
[[ 0.0010016]]
Real prob
[ 0.99774843]
Fake prob
[[ 0.00099208]]
Real prob
[ 0.99808681]
Fake prob
[[ 0.00098283]]
Real prob
[ 0.99750429]
Fake prob
[[ 0.00097372]]
Real prob
[ 0.99725336]
Fake prob
[[ 0.0009648]]
Real prob
[ 0.99821895]
Fake prob
[[ 0.00095601]]
Real prob
[ 0.99812919]
Fake prob
[[ 0.00094738]]
Real prob
[ 0.99838185]
Fake prob
[[ 0.00093891]]
Real prob
[ 0.9957]
Fake prob
[[ 0.0009305]]
Real prob
[ 0.99831307]
Fake prob
[[ 0.00092234]]
Real prob

In [159]:
sample_directory = './figs' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.
batch_size_sample = 256

init = tf.global_variables_initializer()
saver = tf.train.Saver()



with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print 'Loading Model...'
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    print "here"
    Dnewx = discriminator(mnist_data,reuse=True)
    print Dnewx
    D_g1 = sess.run(Dnewx,feed_dict={real_in:mnist_data})
    print D_g1
    

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/model-250.cptk
here
Tensor("d_out_15/Sigmoid:0", shape=(1, 1), dtype=float32)
[[ 0.71755993]]


In [154]:
from PIL import Image, ImageOps
img = Image.open("elephant.jpg")
grayscale = ImageOps.grayscale(img)
grayscale = grayscale.resize((32,32))
grayscale.size
ele_data = np.asarray(grayscale, dtype="float32")
ele_data.shape
ele_data = ele_data.reshape(1,32,32,1)
print ele_data.shape

(1, 32, 32, 1)


In [155]:
from PIL import Image, ImageOps
img = Image.open("mnist.jpg")
grayscale = ImageOps.grayscale(img)
grayscale = grayscale.resize((32,32))
grayscale.size
mnist_data = np.asarray(grayscale, dtype="float32")
mnist_data.shape
mnist_data = mnist_data.reshape(1,32,32,1)
print ele_data.shape

(1, 32, 32, 1)


In [160]:
from PIL import Image, ImageOps
img = Image.open("cat.jpg")
grayscale = ImageOps.grayscale(img)
grayscale = grayscale.resize((32,32))
grayscale.size
cat_data = np.asarray(grayscale, dtype="float32")
cat_data.shape
cat_data = cat_data.reshape(1,32,32,1)
print cat_data.shape

(1, 32, 32, 1)


In [161]:
sample_directory = './figs' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.
batch_size_sample = 256

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print 'Loading Model...'
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    print "here"
    Dnewx = discriminator(cat_data,reuse=True)
    print Dnewx
    D_g1 = sess.run(Dnewx,feed_dict={real_in:cat_data})
    print D_g1
    

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/model-250.cptk
here
Tensor("d_out_16/Sigmoid:0", shape=(1, 1), dtype=float32)
[[ 0.00055689]]


In [162]:
with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print 'Loading Model...'
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    print "here"
    Dnewx = discriminator(ele_data,reuse=True)
    print Dnewx
    D_g1 = sess.run(Dnewx,feed_dict={real_in:ele_data})
    print D_g1
    

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/model-250.cptk
here
Tensor("d_out_17/Sigmoid:0", shape=(1, 1), dtype=float32)
[[ 0.07664267]]


In [163]:
img = Image.open("mnist2.jpg")
grayscale = ImageOps.grayscale(img)
grayscale = grayscale.resize((32,32))
grayscale.size
mnist2_data = np.asarray(grayscale, dtype="float32")
mnist2_data.shape
mnist2_data = mnist2_data.reshape(1,32,32,1)
print mnist2_data.shape

(1, 32, 32, 1)


In [164]:
with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print 'Loading Model...'
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    print "here"
    Dnewx = discriminator(mnist2_data,reuse=True)
    print Dnewx
    D_g1 = sess.run(Dnewx,feed_dict={real_in:mnist2_data})
    print D_g1

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/model-250.cptk
here
Tensor("d_out_18/Sigmoid:0", shape=(1, 1), dtype=float32)
[[ 0.79903275]]
